In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu
from DataLoader import DataLoader
from halfkp import get_halfkp_indeicies
import chess
from chess import Board
import matplotlib.pyplot as plt

import numpy as np

In [2]:
training_generator = DataLoader(64, "train")
val_generator = DataLoader(64, "val", shuffle=False)

In [3]:
lossHistory = []
valHistory = []
class NnueCallbacks(tf.keras.callbacks.Callback):
  def on_batch_end(self, batch, logs=None):
    lossHistory.append(logs['loss'])
    if batch and batch % 1000 == 0:
      pred = self.model.predict(val_generator)
      loss = (pred.flatten() - val_generator.labels.flatten()[0:len(val_generator)*64])**2
      loss = np.sum(loss) / len(loss)
      valHistory.append(loss)

      plt.plot(valHistory)
      plt.savefig("val.png", dpi=200)
      plt.clf()

      plt.plot(lossHistory[20:])
      plt.savefig("loss.png", dpi=200)
      plt.clf()
      
      print(" val_loss : {}".format(loss))

      self.model.save("models/model-{}-{}".format(len(valHistory), int(loss*10000)))

In [4]:
from keras import backend as K
def clipped_relu(x):
    return K.relu(x, max_value=2)

In [5]:
halfAcc = Dense(256, clipped_relu)
i1 = tf.keras.Input((64*64*12,))
i2 = tf.keras.Input((64*64*12,))
acc1 = halfAcc(i1)
acc2 = halfAcc(i2)
layerStack = tf.keras.layers.Concatenate()([acc1, acc2])

h1 = Dense(32, clipped_relu)
h2 = Dense(32, clipped_relu)
out = Dense(1, 'sigmoid')

layerStack = out(h2(h1(layerStack)))

model = tf.keras.Model(inputs=[i1, i2], outputs=layerStack)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse', metrics=['mae'])

2022-04-03 11:58:26.851666: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 11:58:26.856270: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 11:58:26.856753: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 11:58:26.858087: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [ ]:
model = tf.keras.models.load_model("production/")

In [6]:
#625/625 [==============================] - 76s 120ms/step - loss: 0.0710 - mae: 0.1957 - val_loss: 0.0967 - val_mae: 0.2318
model.fit(training_generator, validation_data=val_generator, epochs=1, callbacks=[NnueCallbacks()])

   993/164062 [..............................] - ETA: 17:48 - loss: 0.0844 - mae: 0.2241 val_loss : 0.0811426851906798


2022-04-03 11:31:55.391790: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/model-1-811/assets
  1993/164062 [..............................] - ETA: 19:43 - loss: 0.0818 - mae: 0.2197 val_loss : 0.07851518195065028
INFO:tensorflow:Assets written to: models/model-2-785/assets
  2993/164062 [..............................] - ETA: 19:50 - loss: 0.0803 - mae: 0.2168 val_loss : 0.07652084544559952
INFO:tensorflow:Assets written to: models/model-3-765/assets
  3999/164062 [..............................] - ETA: 19:52 - loss: 0.0792 - mae: 0.2148 val_loss : 0.07614487426376698
INFO:tensorflow:Assets written to: models/model-4-761/assets
  4997/164062 [..............................] - ETA: 19:53 - loss: 0.0784 - mae: 0.2132 val_loss : 0.07463827871565389
INFO:tensorflow:Assets written to: models/model-5-746/assets
  5994/164062 [>.............................] - ETA: 19:48 - loss: 0.0777 - mae: 0.2119 val_loss : 0.07324109045450888
INFO:tensorflow:Assets written to: models/model-6-732/assets
  6997/164062 [>..................

<Figure size 432x288 with 0 Axes>

In [14]:
indicies = get_halfkp_indeicies(Board("r3k1nr/pp2ppbp/n2p2p1/q1p5/3N1P2/1P2P3/PBPP3P/RN1K1B1R w kq - 1 9"))
indicies = indicies.reshape(2, -1)
indicies = (np.array([indicies[0], indicies[0]]), np.array([indicies[1], indicies[1]]))
pred = model.predict(indicies)
print(pred[0])

ValueError: in user code:

    File "/home/levigibson/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "/home/levigibson/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/levigibson/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "/home/levigibson/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1572, in predict_step
        return self(x, training=False)
    File "/home/levigibson/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/levigibson/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 263, in assert_input_compat

In [8]:
pred = model.predict(val_generator[0][0]) * 2 - 1
act = val_generator[0][1] * 2 - 1
for i in range(128):
    print(pred[i], act[i], val_generator.getfen(0)[i])

[-0.3675208] 1.0 r6r/pp2k1pp/2p1Bp2/2b1p3/4P3/8/PPPB1PPP/2KR3R b - - 0 14
[-0.24875295] -0.9999999999999999 r5k1/1b3ppp/p3p3/1pP2n2/8/5NP1/3R1PKP/5R2 b - - 1 24
[-0.28492737] 1.0 r2q1rk1/1b1pbppp/pp2p3/2p3n1/P1BnP1Q1/1P4P1/1BPP1P1P/R4RNK w - - 7 15
[-0.1943143] 1.0 8/p2Q2R1/7p/3pk2B/8/3P4/PPn2P1P/nN3K2 w - - 1 29
[-0.27632397] 1.0 rn1qk2r/pp2bppb/2p1p1np/3pP3/3P4/P2B1N1P/1PPB1PP1/RN1Q1RK1 b kq - 0 10
[-0.47265935] 1.0 rn1q1rk1/pp3b1p/2pb1p2/3p1Bp1/3Pp2P/2P1P3/PP1N1PP1/R1BQ1RK1 b - - 0 15
[-0.42469263] 1.0 2R5/5ppk/8/6p1/2p5/3p4/6rr/3R2K1 w - - 4 45
[-0.42500538] 1.0 1k3r2/p1p5/1p5P/8/6P1/5P2/8/4RRK1 b - - 0 35
[-0.26649833] 1.0 rn2r2k/pp1b2pp/3q1p2/2bPp3/2B4N/2P3B1/P4PPP/R2Q1R1K w - - 6 18
[-0.11016059] 1.0 4r1k1/p4p2/1p1p1bpp/2n5/8/2P2NPP/PP3PB1/R4K2 w - - 0 20
[-0.22793901] 1.0 rn1qkbnr/pp3ppp/2pp4/1B1Pp3/4P1b1/5N2/PPP2PPP/RNBQK2R w KQkq - 0 6
[0.39857173] 1.0 r3k1r1/2q1ppb1/2p5/p3p3/2P1P3/1PNQB1Pb/P4P1P/2KR2R1 b q - 1 22
[0.36378467] 1.0 4rrk1/p4p2/1pp3p1/4Pp1p/1P1P1R2/2q1BQ1P/P4R1K

IndexError: index 64 is out of bounds for axis 0 with size 64

In [7]:
import time
# model.save("models/model-{}".format(int(time.time())))
model.save("production/")

INFO:tensorflow:Assets written to: production/assets
